# Practice Optimizing DataFrames and Processing in Chunks

In this guided project, I practice working with chunked dataframes and optimizing a dataframe's memory usage. I will be working with financial lending data from [Lending Club](https://www.lendingclub.com/), a marketplace for personal loans that matches borrowers with investors.

The Lending Club's website lists approved loans. Qualified investors can view the borrower's credit score, the purpose of the loan, and other details in the loan applications. Once a lender is ready to back a loan, it selects the amount of money it wants to fund. When the loan amount the borrower requested is fully funded, the borrower receives the money, minus the origination fee that Lending Club charges.

I'll be working with a dataset of loans approved from 2007-2011, which you can download from Lending Club's website. I've already removed the desc column to make the system run more quickly.

If I read in the entire data set, it will consume about 67 megabytes of memory. I'm going to imagine that I only have 10 megabytes of memory available throughout this project, so I can practice the concepts.

In [1]:
import pandas as pd
pd.options.display.max_columns = 99

In [2]:
# Reading the dataset and visually checking any quality issues
df = pd.read_csv('loans_2007.csv')
df.head()

/dataquest/system/env/python3/lib/python3.4/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,1077501,1296599.0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-2011,Fully Paid,n,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-1985,1.0,3.0,0.0,13648.0,83.7%,9.0,f,0.00,0.00,5863.155187,5833.84,5000.00,863.16,0.00,0.00,0.00,Jan-2015,171.62,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,1077430,1314167.0,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-2011,Charged Off,n,car,bike,309xx,GA,1.00,0.0,Apr-1999,5.0,3.0,0.0,1687.0,9.4%,4.0,f,0.00,0.00,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,Apr-2013,119.66,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,1077175,1313524.0,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-2011,Fully Paid,n,small_business,real estate business,606xx,IL,8.72,0.0,Nov-2001,2.0,2.0,0.0,2956.0,98.5%,10.0,f,0.00,0.00,3005.666844,3005.67,2400.00,605.67,0.00,0.00,0.00,Jun-2014,649.91,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
3,1076863,1277178.0,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-2011,Fully Paid,n,other,personel,917xx,CA,20.00,0.0,Feb-1996,1.0,10.0,0.0,5598.0,21%,37.0,f,0.00,0.00,12231.890000,12231.89,10000.00,2214.92,16.97,0.00,0.00,Jan-2015,357.48,Apr-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
4,1075358,1311748.0,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,Dec-2011,Current,n,other,Personal,972xx,OR,17.94,0.0,Jan-1996,0.0,15.0,0.0,27783.0,53.9%,38.0,f,461.73,461.73,3581.120000,3581.12,2538.27,1042.85,0.00,0.00,0.00,Jun-2016,67.79,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0


In [3]:
# Reading the first 1000 rows of the dataset and calculating the 
# total memory usage for these rows. I will increase and decrease 
# the number of rows to converge on a memory usage under 5 megabytes
# (to stay on the conservative side)

first_rows = df[:3000]

# Checking the memory usage
first_rows.memory_usage(deep=True).sum()/(2**20)

4.729122161865234

**Answer**
3000 rows seem to be a good load

Next, I will go over to try to understand the column types better while using dataframe chunks.

In [12]:
# Checking how many columns have a numeric type. How many have
# have a string type
chunk_df = pd.read_csv('loans_2007.csv', chunksize=3000)

for chunk in chunk_df:
    numeric_columns = chunk.select_dtypes('number').columns
    string_columns = chunk.select_dtypes('object').columns
    number_numeric_columns = len(numeric_columns)
    number_string_columns = len(string_columns)
                               
    
print(number_numeric_columns, number_string_columns)    

30 22


In [52]:
# Finding how many unique values are on each string column

string_values = {}
chunk_df = pd.read_csv('loans_2007.csv', chunksize=3000)
for chunk in chunk_df:

    string_columns = chunk.select_dtypes('object').columns
    for col in string_columns:
        val_counts = chunk[col].value_counts()
        if col in string_values:
            string_values[col].append(val_counts)
        else:
            string_values[col] = [val_counts]

string_values

{'addr_state': [CA    563
  NY    282
  FL    238
  TX    198
  NJ    133
  GA    122
  IL    117
  PA    103
  VA    100
  OH     91
  WA     82
  MA     81
  NC     79
  MD     70
  AZ     64
  CT     61
  MO     58
  CO     51
  MN     47
  WI     44
  OR     39
  NV     36
  MI     33
  AL     29
  LA     26
  SC     25
  UT     24
  HI     21
  OK     20
  AR     19
  KY     19
  KS     18
  RI     16
  AK     14
  DC     13
  NH     12
  WV     12
  SD     11
  NM      8
  MT      8
  VT      6
  WY      5
  DE      2
  Name: addr_state, dtype: int64, CA    518
  NY    290
  TX    231
  FL    220
  NJ    128
  IL    116
  VA    107
  PA    105
  OH    103
  NC     88
  GA     83
  MA     71
  MD     69
  WA     66
  AZ     64
  MI     59
  CO     58
  CT     54
  MO     51
  MN     45
  WI     45
  NV     41
  AL     37
  SC     34
  KY     34
  LA     33
  KS     32
  OR     29
  UT     27
  OK     25
  AR     19
  HI     17
  RI     16
  NM     12
  DE     12
  WV     11
  DC  

In [87]:
# How many contain values that are less than 50% uniques
unique_per_column = {}
col_small_uniques = []
for col in string_values.keys():
    u_concat = pd.concat(string_values[col])
    u_grouped = u_concat.groupby(u_concat.index).sum()
    unique_per_column[col] = len(u_grouped.index)
    if u_grouped.shape[0] < 50:
        col_small_uniques.append(col)
        print(col, u_grouped.shape[0])
    


application_type 1
home_ownership 5
sub_grade 35
term 2
purpose 14
pymnt_plan 2
grade 7
loan_status 9
initial_list_status 1
verification_status 3
emp_length 11


In [57]:
# Which float columns have no missing values and could be 
# candidates for conversion to the integer type?
chunk_df = pd.read_csv('loans_2007.csv', chunksize=3000)
missing_values = {}
for chunk in chunk_df:
    float_values = chunk.select_dtypes('float').columns
    for col in float_values:
        if col in missing_values:
            missing_values[col] += chunk[col].isnull().sum()
        else:
            missing_values[col] = chunk[col].isnull().sum()

In [58]:
missing_values

{'acc_now_delinq': 32,
 'annual_inc': 7,
 'chargeoff_within_12_mths': 148,
 'collection_recovery_fee': 3,
 'collections_12_mths_ex_med': 148,
 'delinq_2yrs': 32,
 'delinq_amnt': 32,
 'dti': 3,
 'funded_amnt': 3,
 'funded_amnt_inv': 3,
 'inq_last_6mths': 32,
 'installment': 3,
 'last_pymnt_amnt': 3,
 'loan_amnt': 3,
 'member_id': 3,
 'open_acc': 32,
 'out_prncp': 3,
 'out_prncp_inv': 3,
 'policy_code': 3,
 'pub_rec': 32,
 'pub_rec_bankruptcies': 1368,
 'recoveries': 3,
 'revol_bal': 3,
 'tax_liens': 108,
 'total_acc': 32,
 'total_pymnt': 3,
 'total_pymnt_inv': 3,
 'total_rec_int': 3,
 'total_rec_late_fee': 3,
 'total_rec_prncp': 3}

### Total memory across the chunks

In [62]:
# Total memory usage accross the chunks
chunk_df = pd.read_csv('loans_2007.csv', chunksize=3000)
memory = []
for chunk in chunk_df:
    memory.append(chunk.memory_usage(deep=True).sum()/(2**20))
print(sum(memory))

66.2153730392456


In [70]:
chunk_df = pd.read_csv('loans_2007.csv', chunksize=3000)
chunk_df

In [84]:
df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,1077501,1296599.0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-2011,Fully Paid,n,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-1985,1.0,3.0,0.0,13648.0,83.7%,9.0,f,0.00,0.00,5863.155187,5833.84,5000.00,863.16,0.00,0.00,0.00,Jan-2015,171.62,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,1077430,1314167.0,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-2011,Charged Off,n,car,bike,309xx,GA,1.00,0.0,Apr-1999,5.0,3.0,0.0,1687.0,9.4%,4.0,f,0.00,0.00,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,Apr-2013,119.66,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,1077175,1313524.0,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-2011,Fully Paid,n,small_business,real estate business,606xx,IL,8.72,0.0,Nov-2001,2.0,2.0,0.0,2956.0,98.5%,10.0,f,0.00,0.00,3005.666844,3005.67,2400.00,605.67,0.00,0.00,0.00,Jun-2014,649.91,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
3,1076863,1277178.0,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-2011,Fully Paid,n,other,personel,917xx,CA,20.00,0.0,Feb-1996,1.0,10.0,0.0,5598.0,21%,37.0,f,0.00,0.00,12231.890000,12231.89,10000.00,2214.92,16.97,0.00,0.00,Jan-2015,357.48,Apr-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
4,1075358,1311748.0,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,Dec-2011,Current,n,other,Personal,972xx,OR,17.94,0.0,Jan-1996,0.0,15.0,0.0,27783.0,53.9%,38.0,f,461.73,461.73,3581.120000,3581.12,2538.27,1042.85,0.00,0.00,0.00,Jun-2016,67.79,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0


In [89]:
cat_types = {'aplication_type':'category', 'home_ownership':'category',
            'sub_grade':'category', 'purpose':'category', 'grade':'category',
            }

In [88]:
col_small_uniques 

['application_type',
 'home_ownership',
 'sub_grade',
 'term',
 'purpose',
 'pymnt_plan',
 'grade',
 'loan_status',
 'initial_list_status',
 'verification_status',
 'emp_length']

In [91]:
# Determine which string columns you can convert to a numeric type
# if you clean them. For example, the int_rate column is only a 
# string because of the % sign at the end.
memory_2 = []

chunk_df = pd.read_csv('loans_2007.csv', chunksize=3000, dtype=cat_types, 
                       parse_dates=["issue_d", "earliest_cr_line", "last_pymnt_d", "last_credit_pull_d"])
for chunk in chunk_df:
    chunk['int_rate'] = pd.to_numeric(chunk['int_rate'].str.rstrip("%").str.lstrip(" "))
    chunk['revol_util'] = pd.to_numeric(chunk['revol_util'].str.rstrip("%"))
    
    
    memory_2.append(chunk.memory_usage(deep=True).sum()/(2**20))
    
    
print(sum(memory_2))    

42.290218353271484


In [77]:
col_small_uniques

['application_type',
 'home_ownership',
 'sub_grade',
 'term',
 'purpose',
 'pymnt_plan',
 'grade',
 'loan_status',
 'initial_list_status',
 'verification_status',
 'emp_length']

## Next steps for improving project

Create a function that automates as much of the work you just did as possible, so that you could use it on other Lending Club data sets. This function should:

Determine the optimal chunk size based on the memory constraints you provide

Determine which string columns can be converted to numeric ones by removing the % character

Determine which numeric columns can be converted to more space efficient representations